# Wind Farm Test
In this file, one can test certain wind farm layouts and configurations. Some basic functions are demonstrated. With the resulting turbine power outputs and flowfields, one can make their own comparisons between farms.

### Import libraries
Here, the libraries used in this file are imported.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import time
import os
from IPython.display import display, clear_output
from scipy import optimize
from scipy.stats import truncnorm
from itertools import product

# Import custom libraries
import functions as func
import visualization as viz

# Import models
from floris.tools import FlorisInterface as floris

In [18]:
class LES:
    '''
    This class provides tools for handling data obtained with Large Eddy Simulations.
    The output of functions in this class is standardized, so data structures are
    equal between different datasources.
    '''
    def __init__(
        self,
    ):
        pass


    # Get flowfield dataframe
    def get_df_flowfield(
        case_name: str = None,
        location: str = '../LES/',
    ):
        '''
        Get the flowfield dataframe of a LES case.

        Args:
            case_name (str): the name of the case
            location (str): the path to the LES data

        Returns:
            DataFrame: dataframe containing the flowfield data
        '''
        df_flowfield = pd.read_csv(location + case_name + '/' + case_name + '.csv')

        return df_flowfield
    

    def get_gridpoints(
        df_flowfield,    
    ):
        '''
        Get the gridpoints of the LES data

        Args:
            df_flowfield (DataFrame): dataframe containing LES flowfield

        Returns:
            tuple: respectivily the ndarrays of X, Y and Z gridpoints
        '''
        X = np.array(sorted(df_flowfield['Points:0'].unique()))
        Y = np.array(sorted(df_flowfield['Points:1'].unique()))
        Z = np.array(sorted(df_flowfield['Points:2'].unique()))
        
        return X, Y, Z
    

    def get_ABL_params(
        self,
        case_name: str = '1TURB_wd270_ws10_1x_y0_t5',
        location: str = '../LES/',
        z_ref_guess: float = 100,
        U_ref_guess: float = 10,
        alpha_guess: float = 0.12,
    ):  
        '''
        Get the fitted parameters that describe the Atmospheric Boundary
        Layer (ABL) as close as possible below 600m.

        Args:
            case_name (str): the name of the case
            location (str): the path to the LES data
            z_ref_guess (float): initial guess for reference height
            U_ref_guess (float): initial guess for reference velocity
            alpha_guess (float): initial guess for alpha

        Returns:
            dict: containing two arrays with fitted values describing the ABL
        '''
        # Get flowfield
        df_flowfield = self.get_df_flowfield(
            case_name,
            location,
        )

        # Get LES grid points
        X, Y, Z = self.get_gridpoints(df_flowfield)

        # Get U and V profiles of LES simulation at inflow boundary (X = 0)
        U_LES = df_flowfield[df_flowfield['Points:0'] == 0].groupby(['Points:2'])['UAvg:0'].mean().to_numpy()
        V_LES =  df_flowfield[df_flowfield['Points:0'] == 0].groupby(['Points:2'])['UAvg:1'].mean().to_numpy()

        # Get id of points at Z = 600, right before inversion layer starts
        idz, z_value = func.find_nearest(Z, 600)

        # Get U and V profiles from just above Z = 0 to Z = 600, before inversion layer
        cut_start = 2
        Z_cut = Z[cut_start:idz+1]
        U_LES_cut = U_LES[cut_start:idz+1]
        V_LES_cut = V_LES[cut_start:idz+1]

        # Fit streamwise velocity profile parameters
        U_params, _ = sc.optimize.curve_fit(
            func.U_profile, 
            Z_cut, 
            U_LES_cut, 
            p0=[z_ref_guess, U_ref_guess, alpha_guess], 
            bounds=([1, 1, 1e-6], [1e6, 1e6, 1])
        )

        # Fit spanwise velocity profile parameters
        V_params, _ = sc.optimize.curve_fit(
            func.V_profile, 
            Z_cut, 
            V_LES_cut
        )

        return {
            'U_params': U_params,
            'V_params': V_params,
        }


class CCM:
    def __init__(
        self,
        model_params: dict = None,
        input_file: str = 'model_files/CCM/case_initial.yaml',
    ):
        self.input_file = input_file

        if model_params == None:
            model_params = {
                'ad': 0,
                'bd': -0.0018192983887298023,
                'cd': 1.0803331806986867,
                'dd': -0.09040629347972164,
                'alpha': 0.58,
                'beta': 0.077,
                'dm': 1.0,
                'c_s1': 0.0563691592,
                'c_s2': 0.1376631233159683,
                'a_s': 0.3253111149080571,
                'b_s': 0.012031554853652504,
                'a_f': 3.11,
                'b_f': -0.68,
                'c_f': 2.223295807654856,
                'wr_gain': 0.5392489436318193,
                'ma_gain': 1.7431079762733077,
                'wr_decay_gain': 3.207532818500954,
                'ma_decay_gain': 1.7832719494462048,
            }

        self.model_params = model_params

        self.farm = floris(self.input_file)

        self.set_model_params(
            self.model_params
        )
        

    def set_model_params(
        self,
        model_params: dict,
    ):
        '''
        Set model parameters to the CCM model.

        Args:
            model_params (dict): dictionary containing the model parameters
        '''
        # Loop over all model parameters
        for key in model_params.keys():
            found = False

            # Check if in 'gaussm' and if so, apply change
            if key in self.farm.floris.wake.wake_deflection_parameters['gaussm'].keys():
                self.farm.floris.wake.wake_deflection_parameters['gaussm'][key] = model_params[key]
                found = True

            # Check if in 'ccm' and if so, apply change
            if key in self.farm.floris.wake.wake_velocity_parameters['ccm'].keys():
                self.farm.floris.wake.wake_velocity_parameters['ccm'][key] = model_params[key]
                found = True
            
            # Print message if parameter is not found
            if not found:
                print(f'Key named "{key}" not found in either model')
    

    def reinitialize_farm(
        self,
        farm_config: dict,
        model_params: dict = None,
    ):
        # Set model parameters to given or standard parameters
        if model_params == None:
            model_params = self.model_params
        
        # Run twice, since for some reason sometimes things didn't update
        # after running is once (TODO?)
        for _ in range(2):
            # Update farm layout, wind direction and speed
            self.farm.reinitialize(
                layout_x=farm_config['x_ij'].flatten(), 
                layout_y=farm_config['y_ij'].flatten(), 
                wind_directions=farm_config['wind_directions'],
                wind_speeds=farm_config['wind_speeds'],
            )

            # Set model parameters
            if model_params is not None:
                self.set_model_params(model_params)
            
            # Set ABL parameters
            if farm_config['ABL_params'] is not None:
                U_params = farm_config['ABL_params']['U_params']
                V_params = farm_config['ABL_params']['V_params']

                # Set parameters of streamwise velocity profile
                self.farm.floris.flow_field.reference_wind_height = U_params[0]
                self.farm.floris.flow_field.wind_speeds = [U_params[1]]
                self.farm.floris.flow_field.wind_shear = U_params[2]
                
                # Set parameters of spanwise velocity profile
                self.farm.floris.flow_field.a = V_params[0]
                self.farm.floris.flow_field.b = V_params[1]
                self.farm.floris.flow_field.c = V_params[2]
                self.farm.floris.flow_field.d = V_params[3]    


    def get_turbine_powers(
        self,
        farm_config: dict,
    ):  
        # Get yaw and tilt angles flattened and adjust for number of wind conditions
        yaw_angles = farm_config['yaw_ij'].flatten()[None, None]
        tilt_angles = farm_config['tilt_ij'].flatten()[None, None]

        # Create copy of farm so initial farm is not messed up
        farm_copy = self.farm.copy()

        # Calculate wakes
        farm_copy.calculate_wake(
            yaw_angles=yaw_angles,
            tilt_angles=tilt_angles,
        )

        # Get misalignment correction factors. This is not done anymore 
        # in FLORIS itself (TODO?)
        correction_factors = func.get_correction_factor_misalignment(
            yaw_angles,
            tilt_angles,
        )

        # Get total power (need to account for air density and correction factor)
        turbine_powers = farm_copy.get_turbine_powers() * \
            farm_copy.floris.flow_field.air_density * \
            correction_factors

        return turbine_powers

def get_model_class(
    model_name,      
):
    if model_name == 'LES':
        model = LES
    if model_name == 'CCM':
        model = CCM

    return model

In [65]:
def create_layout(
    shape: str = 'rectangular',
    n_x: int = 1,
    n_y: int = 1,
    spacing_x: float = 5,
    spacing_y: float = 5,
    D_rotor: float = 126,
):
    if shape == 'hexagonal':
        # Calculate the vertical and horizontal spacing between hexagon centers
        vertical_spacing = 0.5 * D_rotor * spacing_x
        horizontal_spacing = np.sqrt(3) * D_rotor * spacing_y

        # Lists to store the x and y coordinates of the grid points
        x_i = np.zeros([n_x, n_y])
        y_i = np.zeros([n_x, n_y])

        # Generate the coordinates of the hexagonal grid
        for row in range(n_y):
            for col in range(n_x):
                x = col * horizontal_spacing
                y = row * vertical_spacing

                # Shift every other row horizontally by half the spacing
                if row % 2 == 1:
                    x += horizontal_spacing / 2
                
                x_i[col, row] = x
                y_i[col, row] = y
    else:
        x_i, y_i = np.meshgrid(
            D_rotor * spacing_x * np.arange(0, n_x, 1),
            D_rotor * spacing_y * np.arange(0, n_y, 1),
        )

    return x_i, y_i


class Case:
    def __init__(
        self,
        name: str = 'test',   
        predef_case: dict = {}
    ):  
        # Set name
        self.name = name
        

    def set_layout(
        self,
        shape: str,
        n_x: int = 1,
        n_y: int = 1,
        spacing_x: int = 5,
        spacing_y: int = 5,
        D_rotor: float = 126,
        x_i: list = None,
        y_i: list = None,
    ):
        # Set layout to predefined layout
        if x_i is not None and y_i is not None:
            print('x_i and y_i set to predefined values')
            n_turbines = len(x_i)
        # Set layout according to specifications
        else: 
            x_i, y_i = create_layout(
                shape,
                n_x,
                n_y,
                spacing_x,
                spacing_y,
                D_rotor,
            )

            n_turbines = n_x * n_y

        self.layout = {
            'shape': shape,
            'n_x': n_x,
            'n_y': n_y,
            'x_i': x_i,
            'y_i': y_i,
            'n_turbines': n_turbines,
        }


    def set_conditions(
        self,
        directions: list = np.array([270.]),
        speeds: list = np.array([10.]),
        TI: list = np.array([0.06]),
        ABL_params: dict = None,
    ):
        if type(directions) == int or type(directions) == float:
            directions = np.array([directions])

        if type(speeds) == int or type(speeds) == float:
            speeds = np.array([speeds])

        self.conditions = {
            'directions': directions,
            'speeds': speeds,
            'TI': TI,
            'ABL_params': ABL_params,
        }


    def set_turbines(
        self,
        yaw_i: list,
        tilt_i: list,
        D_rotor_i: list = None,
        CT_i: list = None,
       
    ):  
        # TODO: Add functionality to set CT and D for individual turbs
        
        if D_rotor_i is None:
            D_rotor_i = np.array(self.layout['n_turbines'] * [126])
        # if len(D_rotor_i) == 1:
        #     D_rotor_i = np.array(self.layout['n_turbines'] * [D_rotor_i[0]])
        # if len(yaw_i) == 1:
        #     yaw_i = np.array(self.layout['n_turbines'] * [yaw_i[0]])
        # if len(tilt_i) == 1:
        #     tilt_i = np.array(self.layout['n_turbines'] * [tilt_i[0]])

        self.turbines = {
            'yaw_i': yaw_i,
            'tilt_i': tilt_i,
            'D_rotor_i': D_rotor_i,
            'CT_i': CT_i,
        }


class CaseManager:
    def __init__(
        self,
        name: str = 'Case Manager',
        ref_model = LES,
        ref_data_location: str = '../LES/',
        ref_standard_case: str = '1TURB_wd270_ws10_1x_y0_t5',
    ):
        # Set case manager name
        self.name = name

        # Set reference model, data location and case
        self.ref_model = ref_model
        self.ref_data_location = ref_data_location
        self.standard_ref_case = ref_standard_case

        # Initialize case
        self.cases = {}

        # Get Atmospheric Boundary Layer parameters
        self.standard_ref_ABL_params = ref_model.get_ABL_params(
            ref_model,
            ref_standard_case,
            ref_data_location,
        )

        self.set_case_names()


    def set_case_names(
        self,
    ):
        self.case_names = list(self.cases.keys())

    def load_csv_cases(
        self,
        location: str = '../TouchWind_Optimization_Framework/',
        file_name: str = 'test_cases.csv',
        masks: dict = {},
    ):
        df_cases = pd.read_csv(location + file_name)

        # Select right cases by applying masks
        for mask in masks:
            df_cases = df_cases[df_cases[mask] == masks[mask]].reset_index(drop=True)

        # Create cases
        for name in df_cases['case_name']:
            case_dict = df_cases[df_cases['case_name'] == name].iloc[0]

            case = Case(name)

            x_i, y_i, yaw_i, tilt_i = None, None, None, None

            if not case_dict['equal']:
                x_i, y_i, yaw_i, tilt_i = np.array([]), np.array([]), np.array([]), np.array([])

                for turb in range(case_dict['n_x'] * case_dict['n_x']):
                    x_i[turb] = case_dict[f'x_{turb}']
                    y_i[turb] = case_dict[f'y_{turb}']
                    yaw_i[turb] = case_dict[f'yaw_{turb}']
                    tilt_i[turb] = case_dict[f'tilt_{turb}']
            
            # Set Atmospheric Boundary Layer parameters
            if case_dict['load_ABL'] == 'standard':
                ABL_params = self.standard_ref_ABL_params
            elif case_dict['load_ABL'] == 'ref':
                ABL_params = self.ref_model.get_ABL_params(
                    self.ref_model,
                    name,
                    self.ref_data_location,
                )
            else:
                ABL_params = None

            # Set case layout
            case.set_layout(
                shape=case_dict['shape'],
                n_x=case_dict['n_x'],
                n_y=case_dict['n_y'],
                spacing_x=case_dict['spacing_x'],
                spacing_y=case_dict['spacing_y'],
                D_rotor=case_dict['D_rotor'],
                x_i = x_i,
                y_i = y_i,
            )

            # Set case wind conditions
            case.set_conditions(
                directions=case_dict['wd'],
                speeds=case_dict['U_ref'],
                ABL_params=ABL_params,
            )

            case.set_turbines(
                yaw_i=yaw_i,
                tilt_i=tilt_i,
                D_rotor_i=case_dict['D_rotor'],
            )

            self.cases[name] = case

        self.set_case_names()

In [66]:
# Add new case
test_casemanager = CaseManager(
    name='Test Case Manager',
    ref_model=LES,
    ref_data_location='../LES/',
    ref_standard_case='1TURB_wd270_ws10_1x_y0_t5',
)

# Set masks to select cases from csv-file
masks = {
    'test': 1,
    'optimization': 1,
}

# Load csv files
test_casemanager.load_csv_cases(
    location='../TouchWind_Optimization_Framework/',
    file_name='test_cases.csv',
    masks=masks,
)

In [67]:
for name in test_casemanager.case_names:
    print(test_casemanager.cases[name].conditions)

CCM.get_turbine_powers()

{'directions': 270, 'speeds': 10, 'TI': array([0.06]), 'ABL_params': {'U_params': array([1.04479182e+02, 1.00157564e+01, 8.84845174e-02]), 'V_params': array([  0.98321775,   0.11174136, -45.68489752,  -1.77649479])}}
{'directions': 270, 'speeds': 10, 'TI': array([0.06]), 'ABL_params': None}
{'directions': 270, 'speeds': 10, 'TI': array([0.06]), 'ABL_params': {'U_params': array([1.04458958e+02, 1.00155777e+01, 8.84848804e-02]), 'V_params': array([  0.98328519,   0.1121885 , -45.87263775,  -1.77656633])}}
